<h2><font color="#004D7F" size=6>Módulo 5. Fase de modelado</font></h2>



<h1><font color="#004D7F" size=5>4. Super (lerner)estimadores en Algoritmos ensamblados</font></h1>

<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Manuel Castillo-Cara</font><br>
<font color="#004D7F" size=3>Machine Learning con Python</font><br>

---

<h2><font color="#004D7F" size=5>Índice</font></h2>
<a id="indice"></a>

* [1. Introducción](#section1)
    * [1.1. Librerías](#section11)
    * [1.2. CSV](#section12)
* [2. Super-Estimado en Scikit-learn](#section2)
    * [2.1. Crear lista de modelos](#section21)
    * [2.2. Validación cruzada del Super-Estimador](#section22)
    * [2.3. Evaluación del modelo](#section23)
    * [2.4. Hacer predicciones](#section24)
* [3. Conclusiones](#section3)

In [9]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.core.display import display, HTML
display(HTML("<style>.container{ width:98% }</style>"))

C:\Users\eel20\AppData\Local\Temp\ipykernel_16620\1126351303.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


---

<a id="section1"></a>
# <font color="#004D7F"> 1. Introducción</font>

El problema principal que tenemos como científico de datos es verificar qué modelo es el que mejor comportamiento tiene en nuestro conjunto de datos. Para ello, lo único que nos queda es utilizar la experimentación empírica para probar y descubrir qué modelo de los muchos que hay con diferentes hiperparámetros funciona mejor para su conjunto de datos.

Tenga en cuenta que ya ha ajustado muchos algoritmos diferentes en su conjunto de datos, y algunos algoritmos han sido evaluados muchas veces con diferentes configuraciones. Es posible que tenga muchas decenas o cientos de modelos diferentes de su problema. ¿Por qué no usar todos esos modelos en lugar del mejor modelo del grupo? Esta es la intuición detrás del llamado algoritmo de conjunto ''Súper-Estimador''.

<a id="section11"></a>
## <font color="#004D7F"> 1.1. Librerías</font>

Vamos a declarar algunas librerías generales que ya hemos estado trabajando y que usaremos a lo largo de la sección.

In [6]:
#importing libraries
from math import sqrt
from numpy import hstack
from numpy import vstack
from numpy import asarray
import pandas as pd
#from sklearn.datasets import make_regression
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

<a id="section12"></a>
## <font color="#004D7F"> 1.2. CSV</font>

En esta lección, vamos a utilizar un problema de regresión. Para ello utilizaremos el dataset Boston House Price. Este es un problema de regresión donde todos los atributos son numéricos. 

In [7]:
filename = 'data/housing.csv'
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df_reg = pd.read_csv(filename, delim_whitespace=True, names=names) 
array = df_reg.values
X = array[:,0:13]
y = array[:,13]

Dividiremos los datos para que el 50 por ciento se use para entrenar el modelo y el 50 por ciento se retenga para evaluar el supermodelo final y los modelos base. Fijarse que tenemos 253 por parte de Train/Test

In [8]:
X, X_val, y, y_val = train_test_split(X, y, test_size=0.50)
print('Train', X.shape, y.shape, 'Test', X_val.shape, y_val.shape)

Train (253, 13) (253,) Test (253, 13) (253,)


<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section2"></a>
# <font color="#004D7F"> 2. Super-Lerner(Estimado)en Scikit-learn</font>

<a id="section21"></a>
## <font color="#004D7F"> 2.1. Crear lista de modelos</font>

A continuación, definiremos un conjunto de modelos de regresión diferentes.

La función `get_models()` a continuación define todos los modelos y los devuelve como una lista.

In [11]:
# Crear una lista con los modelos base
def get_models():
    models=list()
    models.append(LinearRegression())
    models.append(ElasticNet())
    models.append(SVR(gamma='scale'))
    models.append(DecisionTreeRegressor())
    models.append(KNeighborsRegressor())
    models.append(AdaBoostRegressor())
    models.append(BaggingRegressor(n_estimators=10))
    models.append(RandomForestRegressor(n_estimators=10))
    models.append(ExtraTreesRegressor(n_estimators=10))
    return models

<a id="section22"></a>
## <font color="#004D7F"> 2.2. Validación cruzada del Super-Estimador</font>

Para este problema utilizaremos un k-fold de 10 particiones. Para cada fold, ajustaremos el modelo en la parte de entrenamiento y haremos predicciones fuera del fold en la parte de validación. Esto se repite para cada modelo y se almacenan todas las predicciones fuera del fold.

Cada predicción fuera del fold será una columna para la entrada del metamodelo. Recopilaremos columnas de cada algoritmo para un fold de los datos, apilando horizontalmente las filas. Luego, para todos los grupos de columnas que recopilamos, apilaremos verticalmente estas filas en un conjunto de datos largo con 253 filas y nueve columnas.

El siguietne Código define la función `get_out_of_fold_predictions()` para un conjunto de datos de Train y una lista de modelos; devolverá el conjunto de datos de entrada y salida necesarios para entrenar el metamodelo.

In [18]:
# collect out of fold predictions form k-fold cross validation
def get_out_of_fold_predictions(X, y, models):
    meta_X, meta_y = list(), list()
    #Definir del k-fold
    kfold = KFold(n_splits=10, shuffle=True)
    # Enumeramos las particiones
    for train_ix, test_ix in kfold.split(X):
        fold_yhats = list()
        #obtenemos los datos
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        meta_y.extend(test_y)
        #Ajustes y predicciones de cada submodelo
        for model in models:
            model.fit(train_X, train_y)
            yhat = model.predict(test_X)
            #Almacenamiento de cada dcolumna
            fold_yhats.append(yhat.reshape(len(yhat), 1))
        #Almacenar el fold e yhats como columnas
        meta_X.append(hstack(fold_yhats))
    return vstack(meta_X), asarray(meta_y)

<div class="alert alert-block alert-info">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Documentación oficial de la clase [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).
</div> 

<a id="section23"></a>
## <font color="#004D7F"> 2.3. Evaluación del modelo</font>

<a id="section231"></a>
### <font color="#004D7F"> 2.3.1. Preparar el conjunto de datos</font>

Entonces podemos llamar a la función para obtener los modelos y la función para preparar el conjunto de datos del metamodelo

In [19]:
# get models
models = get_models()
# get out of fold predictions
meta_X, meta_y = get_out_of_fold_predictions(X, y, models)


<a id="section232"></a>
### <font color="#004D7F"> 2.3.2. Ajuste de los modelos base</font>

A continuación, podemos ajustar todos los modelos base en todo el conjunto de datos de entrenamiento.

In [20]:
# fit all base models on the training dataset
def fit_base_models(X, y, models):
    for model in models:
        model.fit(X, y)
        

<a id="section233"></a>
### <font color="#004D7F"> 2.3.3. Ajuste del Meta-modelo</font>

Luego, podemos ajustar el metamodelo en el conjunto de datos de entrenamiento. En este caso, utilizaremos un modelo de regresión lineal como metamodelo.

In [21]:
# fit a meta model
def fit_meta_model(X, y):
    model =LinearRegression()
    model.fit(X, y)
    return model

<a id="section234"></a>
### <font color="#004D7F"> 2.3.4. Evaluación de los modelos base</font>

A continuación, podemos evaluar los modelos base en el conjunto de datos de reserva.

In [25]:
# evaluate a list of models on a dataset
def  evaluate_models(X, y, models):
    for model in models:
        yhat = model.predict(X)
        mse = mean_squared_error(y, yhat)
        print('%s: RMSE %.3f' % (model.__class__.__name__, sqrt(mse)))

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section24"></a>
## <font color="#004D7F"> 2.4. Hacer predicciones</font>

Finalmente, usaremos el Super-Estimador (base y metamodelo) para hacer predicciones sobre el conjunto de datos y evaluar el rendimiento del enfoque.

La función `super_learner_predictions()` utilizará el metamodelo para hacer predicciones para nuevos datos. Podemos llamar a esta función y evaluar los resultados

In [26]:
# make predictions with stacked model
def super_lerner_prediction(X, models, meta_model):
    meta_X = list()
    for model in models:
        yhat = model.predict(X)
        meta_X.append(yhat.reshape(len(yhat),1))
    meta_X = hstack(meta_X)
    return meta_model.predict(meta_X)

Podemos evaluar esta función y evaluar el modelo. Para ello terminamos de insertar el código.

In [27]:
##ya se hizo

## get models
#models = get_models()
## get out of fold predictions
#meta_X, meta_y = get_out_of_fold_predictions(X, y, models)

##hasta aqui 

# fit base models
fit_base_models(X, y, models)
# fit the meta model
meta_model = fit_meta_model(meta_X, meta_y)
# evaluate base models
evaluate_models(X_val, y_val, models)
# evaluate meta model
yhat = super_lerner_prediction(X_val, models, meta_model)
print('Super Lerner: RMSE %.3f' % (sqrt(mean_squared_error(y_val, yhat))))

LinearRegression: RMSE 4.669
ElasticNet: RMSE 5.050
SVR: RMSE 8.356
DecisionTreeRegressor: RMSE 4.497
KNeighborsRegressor: RMSE 6.782
AdaBoostRegressor: RMSE 3.780
BaggingRegressor: RMSE 3.630
RandomForestRegressor: RMSE 3.627
ExtraTreesRegressor: RMSE 3.439
Super Lerner: RMSE 3.564


<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section3"></a>
# <font color="#004D7F"> 3. Conclusiones</font>

Al ejecutar el ejemplo, primero se informa la forma del conjunto de datos preparado, luego la forma del conjunto de datos para el metamodelo.

A continuación, el rendimiento de cada modelo base se informa en el conjunto de datos de espera y, finalmente, el rendimiento del Súper-Estimador en el conjunto de datos de espera.

Sus resultados específicos serán diferentes dada la naturaleza estocástica del conjunto de datos y los algoritmos de aprendizaje. Intenta ejecutar el ejemplo varias veces.

En este caso, podemos ver que los modelos de conjunto funcionan bien en el conjunto de datos y los algoritmos lineales y no lineales no tan bien.

También podemos ver que el Súper-Estimador superó a todos los modelos base

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>